# Load environment variables

1, Create a `.env` file from the root directory of the project

2, Add the GEMINI_API_KEY variable

In [1]:
from dotenv import load_dotenv
from typing import List

load_dotenv()

True

# Sample

## Gemini call

Calling the Gemini model to generate an output with a calendar event schema using `call_gemini_calendar_event()`

In [39]:
from agentic_workspace.llm_clients.gemini_calls import call_gemini_calendar_event
from agentic_workspace.models.calendar_event import CalendarEventModel
from agentic_workspace.llm_clients.promt_template import calendar_event_prompt


prompt = calendar_event_prompt(
    objective="Create 3 test events consecutively",
    constraints="Time zone Ho Chi Minh, description in detail"
    )

response : List[CalendarEventModel] = call_gemini_calendar_event(prompt)

for event in response:
    event = event.model_dump()

    print("\n---- ")
    print("About event: ", event['text'])
    print("Summary: ", event['summary'])
    print("Description: ", event['description'])
    print("Start: ", event['start'])
    print("End: ", event['end'])



---- 
About event:  Meeting with Team A to discuss project progress
Summary:  Team A Meeting
Description:  Discuss project progress, review tasks, and plan next steps. Time zone: Ho Chi Minh
Start:  {'dateTime': '2025-04-06T11:00:00+07:00', 'timeZone': 'Asia/Ho_Chi_Minh'}
End:  {'dateTime': '2025-04-06T12:00:00+07:00', 'timeZone': 'Asia/Ho_Chi_Minh'}

---- 
About event:  Lunch with John at local restaurant
Summary:  Lunch with John
Description:  Have lunch with John and discuss potential collaboration opportunities. Time zone: Ho Chi Minh
Start:  {'dateTime': '2025-04-06T12:30:00+07:00', 'timeZone': 'Asia/Ho_Chi_Minh'}
End:  {'dateTime': '2025-04-06T13:30:00+07:00', 'timeZone': 'Asia/Ho_Chi_Minh'}

---- 
About event:  Prepare presentation for the upcoming conference
Summary:  Presentation Preparation
Description:  Work on presentation slides, rehearse the speech, and finalize the content. Time zone: Ho Chi Minh
Start:  {'dateTime': '2025-04-06T14:00:00+07:00', 'timeZone': 'Asia/Ho_Chi

# Call google calendar API

# Insert event

In [40]:
from agentic_workspace.tools.google_calendar.actions import GoogleCalendar

with GoogleCalendar(calendar_id = 'primary') as calendar_client:
    for _event in response:
        try:
            _event_formated = _event.model_dump()
            _event_formated['colorId'] = '5'

            event = calendar_client.insert_event(event=_event_formated)

            print(f"Event created: {event['summary']} --- Event URL: {event['htmlLink']}")

        except Exception as e:
            print(f'An error occurred: {e}')


Event created: Team A Meeting --- Event URL: https://www.google.com/calendar/event?eid=aDk1dmhobHN0c282dmNzMThwcnNqY2M2dDggdG90aGFuaDFmZWIzLnF1aW5uQG0
Event created: Lunch with John --- Event URL: https://www.google.com/calendar/event?eid=MjJtZmNocWVwdWwwYWc3NDRodTM0YzJlNWcgdG90aGFuaDFmZWIzLnF1aW5uQG0
Event created: Presentation Preparation --- Event URL: https://www.google.com/calendar/event?eid=ZWxucDYyYmtvbDd0NjQ3ODJubDZxbm40aG8gdG90aGFuaDFmZWIzLnF1aW5uQG0


## List event

In [28]:
from agentic_workspace.tools.google_calendar.actions import GoogleCalendar

with GoogleCalendar() as calendar_client:
    events = calendar_client.list_event(timeMin='2025-01-01T00:00:00Z', maxResults=1)

    print("----\nList event found:")
    if not events:
        print('No upcoming events found.')
    else:
        for event in events:
            start = event['start'].get('dateTime', event['start'].get('date'))
            print(f"{event['id']} - {start} - {event['summary']}")

----
List event found:
3o1sp2ne88u1vot8m1t0pil3p0 - 2025-03-24T09:00:00+07:00 - WP Sửa chữa lưu động - Thêm trường, Schema, Business Schema


## Get event

In [31]:
from agentic_workspace.tools.google_calendar.actions import GoogleCalendar

with GoogleCalendar() as calendar_client:
    event = calendar_client.get_event('3o1sp2ne88u1vot8m1t0pil3p0')

    print("----\nList event found:")
    if not event:
        print('No upcoming events found.')
    else:
        start = event['start'].get('dateTime', event['start'].get('date'))
        print(f"{event['id']} - {start} - {event['summary']}")

----
List event found:
3o1sp2ne88u1vot8m1t0pil3p0 - 2025-03-24T09:00:00+07:00 - WP Sửa chữa lưu động - Thêm trường, Schema, Business Schema


# Update event

In [42]:
from agentic_workspace.tools.google_calendar.actions import GoogleCalendar

with GoogleCalendar() as calendar_client:
    try:
        event = calendar_client.update_event(event_id="3o1sp2ne88u1vot8m1t0pil3p0", event={"colorId": "2"})

        print(f"Event updated: {event['summary']} --- Event URL: {event['htmlLink']}")

    except Exception as e:
        print(f'An error occurred: {e}')

Event updated: WP Sửa chữa lưu động - Thêm trường, Schema, Business Schema --- Event URL: https://www.google.com/calendar/event?eid=M28xc3AybmU4OHUxdm90OG0xdDBwaWwzcDAgdG90aGFuaDFmZWIzLnF1aW5uQG0


# Delete event

In [ ]:
from agentic_workspace.tools.google_calendar.actions import GoogleCalendar

with GoogleCalendar() as calendar_client:
    # insert an event
    inserted_event = calendar_client.insert_event({
        'summary': 'Test Event',
        'description': 'This is a test event',
        'start': {
            'dateTime': '2023-05-01T23:00:00+07:00',
            'timeZone': 'America/New_York'
        },
        'end': {
            'dateTime': '2023-05-01T23:00:00+07:00',
            'timeZone': 'America/New_York'
        }
    })


    # Delete event
    response = calendar_client.delete_event(inserted_event['id'])

    print(response)